<a href="https://colab.research.google.com/github/Bisheshsingh/AI-ML/blob/main/Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image
from PIL import Image
import matplotlib.pyplot as plt

# Model

In [2]:
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.select = ['0', '5', '10', '19', '28'] 
        self.vgg = models.vgg19(pretrained=True).features
        
    def forward(self, x):
        features = []
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features


# Dataset

In [6]:
def load_img(name):
    path=description['project dir']+'/'+name
    img=Image.open(path)
    img=description['loader'](img).unsqueeze(0)
    img=img.to(description['device'])
    return img


inp_img=load_img('36.jpeg')
st_img=load_img('37.jpeg')
generated=inp_img.clone().requires_grad_(True)
st_img.shape

torch.Size([1, 3, 356, 356])

# Description

In [4]:
description={
    'project name' : 'CIFAR10_classification',
    'project dir' : os.getcwd(),
    'train_batch' : 125,
    'test_batch' : 125,
    'loader' : transforms.Compose([
              transforms.Resize(size=356),
              transforms.ToTensor(),
              #transforms.Normalize()
    ]),
    'device' : 'cpu'
}

# Implementation

In [7]:
def loss_style(gen,inp,st,al,bt):
    inp_loss=0
    st_loss=0
    for g,i,s in zip(gen,inp,st):
        bs,ch,h,w=g.shape
        inp_loss+=torch.mean((g-i)**2)
        G=g.view(ch,h*w).mm(g.view(ch,h*w).t())
        A=s.view(ch,h*w).mm(s.view(ch,h*w).t())

        st_loss+=torch.mean((G-A)**2)
    total_loss=al*inp_loss+bt*st_loss
    return total_loss


In [8]:
steps=6000
alpha=1
beta=0.01
learn_rate=1e-2

model=VGGNet().to(description['device'])
optimizer=optim.Adam([generated],lr=learn_rate)

for step in range(steps):
    gen_features=model(generated)
    inp_features=model(inp_img)
    st_features=model(st_img)
    total_loss=loss_style(gen_features,inp_features,st_features,alpha,beta)
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    if step%200==0:
      print(total_loss)
      save_image(generated,'gen.jpg')



Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

tensor(270314.6562, grad_fn=<AddBackward0>)


KeyboardInterrupt: ignored